# Cartographie avec d3

D3 contient un grand nombre de fonctions qui facilitent la création de cartes interactives. On peut également intégrer d3 avec Leaflet sans grand problème. Nous allons voir ici les bases de la cartographie thématique avec d3. Par contre, nous devons d'abord faire un petit détour par des aspects plus généraux d'optimisation de la cartographie interactive.

## 1. Faire mieux que du GeoJSON

Traditionnellement, des couches SIG vectorielles ont été stockées dans des fichiers Shape. Ce format n'étant pas très bien adapté à la cartographie sur le Web car il s'agit d'un format binaire qui en plus est réparti sur au moins 3 fichiers différents. Pour cette raison, le format GeoJson a vu le jour qui peut être considéré comme étant l'équivalent des fichiers Shape pour la cartographie sur le Web.

Un fichier GeoJSON a la structure suivante:

```json
{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": { "type": "Point", "coordinates": [36.706988, 55.664097] },
            "properties": { "name": "Khar Turan National Park" }
        },
        {
            "type": "Feature",
            "geometry": { "type": "Point", "coordinates": [37.547378, 56.381516] },
            "properties": { "name": "Golestan National Park" }
        }
    ]
}
```

Un fichier GeoJSON est facile à lire et à éditer, et avec QGIS ou des outils comme [ogr2ogr](http://www.gdal.org/ogr2ogr.html), il est facile d'en créer un à partir d'un fichier Shape p.ex.

Par contre, le problème principal des fichiers GeoJson est leur taille relativement importante même pour des régions relativement petites. C'est un problème lorsqu'on doit transférer ces fichiers par le Web. La taille importante des fichiers est dû à plusieurs raisons:

- Beaucoup d'éléments sont répétés, comme par exemple `"type": "Feature"`, `"geometry": { "type": "point", ... }`, ou encore les noms des attributs. L'exemple ci-dessus pourrait être écrit comme suit sans perdre de l'information:

        point                name
        36.706988,55.664097  Khar Turan National Park
        37.547378,56.381516  Golestan National Park
  
  ce qui fait 98 octets contre 459 octets, donc un _taux de compression_ de plus de 450%!
  

- La précision des coordonnées est souvent plus élevée que ce dont on a besoin pour la cartographie interactive. On n'a pas besoin d'avoir une précision au centimètre p.ex. On peut donc souvent enlever quelques décimales, style:

        point          name
        36.707,55.664  Khar Turan National Park
        37.547,56.382  Golestan National Park
  
  ce qui nous permet de réduire encore le volume de données (on appelle cette procédure __«quantization»__)


- Notamment pour les couches des polygones de limites administratives, très utilisées dans les cartes choroplèthes, quasiment chaque arrête de polygone est répétée. En effet, chaque unité administrative est définie par le polygone complet, et l'unité voisine répète la frontière commune. Ce format de données est appelé le __format spaghetti__, contrairement au __format topologique__ qui évite cette redondance. Le format topologique a par ailleurs l'avantage d'éviter des problèmes topologiques où la frontière commune n'est pas définie exactement de la même manière. Si nous changeons la façon de stocker nos géométries pour un format topologique, nous pouvons donc économiser presque la moitié des coordonnées...


- Beaucoup de couches vectorielles sont très peu généralisées et donc plus détaillées que nécessaire pour une cartographie thématique. En plus, le format spaghetti rend l'utilisation d'algorithmes automatiques de simplification difficile, en raison du risque d'introduire des erreurs topologiques. L'utilisation d'un format topologique facilite la __simplification__ automatique. Ceci permet également d'adapter le niveau de généralisation au niveau de zoom plus facilement.


- Dans certaines situations, nous n'avons pas besoin de l'entier du fichier des géometries, p.ex. si on visualise uniquement une partie de la région lors d'un zoom. Il suffirait dans ce cas de transmettre uniquement une sous-région du fichier initial.


- Les fichiers JSON transmis contiennent du texte ASCII, et contient souvent uniquement environ 60 à 70 caractères différents, comparé aux 256 caractères possibles. Une traduction en format binaire optimisé pourrait donc optimiser presque deux tiers du volume de données transmis.

Pour palier à ces différents problèmes, il y a deux approches qui ont vu le jour ces dernières années:

1. le [format TopoJson](https://github.com/topojson/topojson) de Mike Bostock, un des principaux développeurs de d3
2. les [Vector Tiles](https://www.mapbox.com/vector-tiles/) de Mapbox

Le format TopoJson adresse tous les points mentionnés ci-dessus sauf les deux derniers, tandis que les Vector Tiles focalisent plus sur ces deux derniers points. Nous allons couvrir ici uniquement le format TopoJson.


## 2. TopoJSON

TopoJSON est un format topologique qui compose un polygone à partir d'une séries de lignes (les _arcs_). Donc au lieu d'énumérer les points du polygone, il y a d'un côté une liste avec tous les arcs existants dans une ou plusieurs couches vectorielles, et de l'autre côté une liste d'arcs qui composent un polygone.

Un fichier TopoJson peut facilement intégrer plusieurs couches, comme p.ex. les couches des communes ensemble avec les districts et les cantons. Ceci fait du sens vu que les arcs à utiliser sont les mêmes.

Les formats topologiques ne sont pas nouveaux dans les SIG. En effet, les anciens logiciels SIG comme GRASS GIS et ARC/INFO avaient des formats topologiques pour les données vectorielles. ARC/INFO est devenu, après la version 7, ArcGIS 8, mais sans intégrer le format topologique, même si le mot _arc_ restait dans le nom...

En plus d'implémenter un format topologique, TopoJson intègre également quelques autres optimisations, comme la réduction du nombre des décimales. Le résultat est un format beaucoup plus compact que le GeoJSON, et qui se prête mieux pour d'autres optimisations, particulièrement au niveau de la généralisation.

Le blog de Mike Bostocks contient un excellent article sur le format TopoJSON et le raisonnement au niveau de la topologie qui est à la base. [[How To Infer Topology](https://bost.ocks.org/mike/topology/)]

`geo2topo` est un logiciel de commande de ligne développé par Mike Bostocks qui permet de traduire un ou plusieurs fichiers GeoJSON en format TopoJSON. Il peut être combiné avec d'autres logiciels de ligne de commande pour couvrir toute la chaîne de travail de la cartographie interactive (voir l'article _[«Command-Line Cartography»](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c)_ sur le blog de Mike Bostocks).

Pour installer les logiciels nécessaires pour TopoJSON, le plus simple est de l'installer avec NPM:

```shell
npm install -g topojson
```

Si NPM n'est pas installé, il est également possible de [le télécharger](https://github.com/topojson/topojson/releases/latest). Ou mieux, on peut installer NPM (livré avec [Node JS](https://nodejs.org)), p.ex. avec Homebrew sur le Mac:

```shell
brew install node
```

Pour l'exercice, nous allons utiliser les couches des communes suisses, ensemble avec les lacs, disponibles en fichier GeoJSON:

- [Communes suisses 2017](https://raw.githubusercontent.com/christiankaiser/geovis2/master/data/communes-vec200-2017.geojson) (Vector-200, source Swisstopo, taille: 5.42 Mo)
- [Lacs suisses](https://raw.githubusercontent.com/christiankaiser/geovis2/master/data/vec200-lacs.geojson) (même source, taille: 284 Ko)

Notamment le fichier des communes est trop gros pour être chargé à travers le Web, et même compressé, il fait encore 1.2 Mo (la compression est généralement appliquée par défaut par les serveurs Web, sans que l'on remarque quelque chose).



La __conversion des fichiers GeoJSON en TopoJSON__ se fait pour toutes les couches à la fois. Ceci est important, car les lacs et les communes partagent les contours. La commande suivante permet de le faire:

```bash
geo2topo communes=communes-vec200-2017.geojson lacs=vec200-lacs.geojson > communes-lacs-topo.json
```

Ceci crée le fichier TopoJSON, qui a une taille de 2.5 Mo, sans avoir appliqué une simplification ou autre modification.

L'indication `communes=...` et `lacs=...` permet de donner un nom à chaque couche à l'intérieur du fichier TopoJSON. Nous pouvons choisir librement ce nom.

La taille du fichier après compression est encore environ 800 Ko, ce qui est déjà bien. Mais nous pouvons encore __simplifier les géométries__ un peu, nous n'avons pas besoin de tous les détails pour une carte à l'échelle de la Suisse. Ceci se fait avec la commande __toposimplify__:

```bash
toposimplify -p 100 -f < communes-lacs-topo.json > communes-lacs-simpl-topo.json
```

ce qui réduit la taille du fichier encore une fois un peu. Le paramètre `-p 100` élimine toutes les entités qui sont plus petites que 100 m<sup>2</sup>. `-f` élimine des éventuels arcs non utilisés.

On peut encore __réduire la précisions des coordonnées__:

```bash
topoquantize 1e4 < communes-lacs-simpl-topo.json > communes-lacs-quant-topo.json
```

Le paramètre de qualité est dans ce cas `1e4` (on peut aussi écrire `10000` à la place). Une valeur plus élevée indique une précision supérieure, mais des tailles de fichier plus grands.

Après cette opération, la taille du fichier GeoJSON est encore environ 1.2 Mo, ou 320 Ko après compression. Ce qui est déjà très acceptable...

__Astuce:__ on peut enchaîner les trois étapes en une seule commande de Terminal:

```bash
geo2topo communes=communes-vec200-2017.geojson lacs=vec200-lacs.geojson | toposimplify -p 100 -f | topoquantize 1e4 > communes-lacs-quant-topo.json
```

Ceci fonctionne grâce à l'opérateur `|` qui prend la sortie d'une commande comme entrée à la commande suivante.

On peut encore réduire la taille du fichier, en enlevant des attributs non utilisés, et en modifiant les noms des attributs (p.ex. au lieu de `POP_DENS`, utiliser `DNS`, ce qui permet d'enlever 5 caractères pour chacune des 2500 communes). Ainsi, on arrive à une taille d'environ 1 Mo pour le fichier TopoJSON, ou moins de 300 Ko après compression. À comparer avec les presque 5.5 Mo au départ!

Pour __contrôler le résultat de ces étapes__, il est possible de convertir le fichier TopoJSON à nouveau en fichier GeoJSON:

```bash
topo2geo communes=cmnes.geojson lacs=lacs.geojson < communes-lacs-quant-topo.json
```

ce qui crée les deux fichiers `cmnes.geojson` et `lacs.geojson`.

Pour l'instant, nous avons uniquement les limites des communes et les lacs. On peut encore rajouter les __limites des cantons__ par exemple, car nous avons un attribut qui associe chaque commune au canton. L'extraction fonctionne avec la commande __`topomerge`__:

```bash
topomerge -k 'd.properties.KANTONSNUM' cantons=communes < communes-lacs-quant-topo.json > communes-cantons-lacs-quant-topo.json
```

Le paramètre `-k 'd.properties.KANTONSNUM'` indique l'attribut qui doit être utilisé pour la fusion des géométries; `d` étant la commune (c'est défini par `topomerge`).

Notez aussi que la taille du fichier n'a quasiment pas changé, malgré le fait que nous avons ajouté tous les cantons!

Nous sommes maintenant prêts à faire une carte avec notre fichier TopoJSON.

## 3. Carte SVG

Un exemple relativement simple de carte SVG avec d3 et TopoJSON se trouve ici: [github.com/christiankaiser/d3-topojson-choropleth](https://github.com/christiankaiser/d3-topojson-choropleth).

Cet exemple montre une carte choroplèthe de la proportion de femmes de 20-35 ans célibataires, par commune suisse:

- la carte utilise 3 couches de géométries: les communes, les limites des cantons, et les lacs
- les données thématiques sont chargées séparément, depuis un fichier TSV
- la mise en classe est faite avec la méthode de Jenks, en utilisant [Classybrew](https://github.com/tannerjt/classybrew) (plus particulièrement le [fork ici](https://github.com/christiankaiser/classybrew)).
- la palette des couleurs est intégrée dans Classybrew, et vient de [Colorbrewer](http://colorbrewer2.org).

Le résultat final se présente comme suit (sans titre, légende, etc. pour le moment):  
![](figures/carte-d3.png)

Nous allons regarder ici uniquement les éléments clés du code de la carte. Une fois que ces éléments sont claires, le code est relativement facile à comprendre.

### 3.1 Convertir une géométrie GeoJSON en path SVG

d3 propose la fonction __`d3.geoPath()`__ pour convertir une géométrie GeoJSON en path SVG qui peut être utilisé pour dessiner le polygone.

_(Le code ci-dessous ne fonctionne pas tel quel, il doit être adapté et inséré au bon endroit dans le vrai code.)_  
Ceci se fait en deux étapes. Tout d'abord on crée un objet de conversion comme suit:

```javascript
var path = d3.geoPath();
```

Et puis, il est appliqué à l'attribut `d` de l'élément SVG `path`:

```javascript
d3.select('svg')
  .selectAll('path')
  .data(geojson.features)
  .enter()
  .append('path')
  .attr('d', path);
```

Dans le code ci-dessus, on associe tous les _features_ d'un objet _geojson_ à un `path` SVG, et puis on définit l'attribut `d` sur la dernière ligne comme étant le `path` créé auparavant (il s'agit d'une fonction qui reçoit automatiquement le _feature_ GeoJSON).

La __conversion de TopoJSON en path SVG__ n'est pas possible directement. On doit d'abord convertir le TopoJSON en GeoJSON, ce qui est expliqué ci-dessous.

### 3.2 Convertir une géométrie TopoJSON en GeoJSON

La conversion de TopoJSON en GeoJSON nécessite d'inclure la libraire TopoJSON dans le fichier HTML:

```html
<script src="https://d3js.org/topojson.v2.min.js"></script>
```

Par la suite, la conversion du TopoJSON en GeoJSON est très simple, voici l'exemple pour la couche des communes dans le fichier TopoJSON (rappel: le fichier TopoJSON peut contenir plus qu'une couche):

```javascript
var geojson = topojson.feature(data, data.objects.communes)
```

où `data` est le contenu du fichier TopoJSON. Les _features_ individuels se trouvent dans `geojson.features` (utilisé dans le code de la section précédente).

### 3.3 Charger plusieurs fichiers externes à la fois

Pour notre carte, nous utilisons deux fichiers externes:
- le fichier TopoJSON avec les géométries
- le fichier TSV avec les données statistiques

Nous pouvons charger ces fichiers avec d3 de manière asynchrone avec `d3.json` respectivement `d3.tsv`. Le chargement asynchrone permet de charger le fichier et sans attendre continuer l'exécution du reste du code. Une fois que le fichier est chargé, une fonction est appelée automatiquement.

Le problème est qu'on ne sait pas quand la fonction sera appelée, et si on a plusieurs fichiers à charger, dans quel ordre. Si on a besoin du contenu des deux fichiers, il faut contrôler à chaque fois si l'autre fichier est déjà chargé etc., ce qui est assez lourd à implémenter. d3 propose un mécanisme pour palier à ce problème, à travers l'implémentation d'une __queue d'attente__. Ceci fonctionne grosso modo comme suit:

```javascript
d3.queue()
  .defer(d3.json, 'url du fichier TopoJSON')
  .defer(
    d3.tsv, 
    'url du fichier TSV', 
    function(d){ console.log('fonction callback TSV'); }
  )
  .await(fonctionCallbackFinale);
```

où la fonction `fonctionCallbackFinale` est appelée uniquement une fois que les deux fichiers ont été complétement chargés. La fonction prend la structure suivante:

```javascript
function fonctionCallbackFinale(error, dataJSON, dataTSV){
    if (error) throw error;
    
    console.log('Fichiers ont été chargés');
    console.log(data);
    ...
}
```

Le nom `fonctionCallbackFinale` peut évidement changer en fonction de nos envies.

On peut observer que la fonction `defer` utilisé en haut peut accepter une fonction de callback, p.ex. pour stocker les données quelque part, ou les traiter avant de continuer.

### 3.4 Projection des coordonnées

Les coordonnées dans le fichier TopoJSON ou GeoJSON sont des coordonnées du système de référence spatiale (SRS), et non des coordonnées SVG. Il faut donc projeter les coordonnées du SRS en coordonnées locales SVG.

Ceci n'est pas une projection dans le sens cartographique, mais plutôt une transformation des coordonnées. 

Par contre, si les coordonnées sont en lat/long (un SRS géographique, p.ex. WGS84), on doit faire une projection cartographique avant la transformation des coordonnées en coordonnées locales SVG.

Même si d3 peut s'occuper de la projection cartographique, nous n'allons pas traiter ce cas ici, car nos géométries sont en CH1903/LV03 (EPSG:21781), ce qui est déjà un SRS projeté. Nous avons grosso modo les possibilités suivantes:

- __Transformer nos coordonnées dans le fichier TopoJSON.__ Ceci a l'avantage de limiter les calculs nécessaires dans le navigateur, ce qui est important si on a beaucoup de géométries. Le désavantage est que la taille de la carte SVG doit être connue en avance.

- __Transformer les coordonnées avec d3 dans le navigateur.__ d3 propose quelques fonctions pour faire cette transformation, ou nous pouvons écrire notre propre fonction de conversion. L'avantage de cette approche est la grande flexibilité qu'elle offre, le désavantage est le nombre relativement important de calculs nécessaires dans le navigateur, ce qui peut ralentir l'affichage de la carte.

- __Utiliser une transformation SVG sur un élément SVG `g`__ qui contient l'ensemble des géométries. En effet, pour transformer les coordonnées originales des géométries, il suffit de faire une mise à l'échelle et une translation. Nous pouvons donc simplement utiliser l'attribut SVG `transform` ensemble avec les bons paramètres. Cette approche nous offre également une grande flexibilité, elle est relativement simple à mettre en oeuvre, et le calcul est faite par l'implémentation interne au navigateur de l'affichage SVG. Les navigateurs bien optimisés utiliseront la carte graphique pour faire cette transformation. Le désavantage est que p.ex. les épaisseurs de lignes sont également affectés par la transformation; on doit donc indiquer l'épaisseur d'une ligne non plus en pixels à l'écran, mais en mètres sur le terrain...

Nous allons regarder ici uniquement la 3ème option, les autres options seront traitées plus tard.

La transformation SVG consiste en une mise à l'échelle et une translation, ce qui peut être exprimé par une matrice de transformation affine comme suit:

```
matrix(scaleX 0 0 scaleY dx dy)
```

où $dx$ et $dy$ correspondent à la translation. Il faut encore noter que le paramètre `scaleY` doit être négatif, car l'axe de Y va du haut vers le bas en SVG, tandis que c'est l'inverse pour le SRS. Et finalement, la valeur absolue de `scaleX` et `scaleY` doit être la même, sinon il y a distortion de la carte.

Le résultat peut être exprimé en pseudo-code comme suit:

```javascript

// Calculer les facteurs de mise à l'échelle
var scaleX = width_svg / width_carte,
    scaleY = height_svg / height_carte;

// On sélectionne l'échelle qui permet d'afficher tout sans distortion:
var scale = Math.min(scaleX, scaleY);

// Calcul de la translation
var dx = -1 * scale * coord_minx_carte,
    dy = scale * coord_miny_carte + height_svg;
// Notice: la part `+ height_svg` est à cause de l'inversion de l'axe des y

// Et finalement créer un groupe SVG avec le transform
var map = d3.select('svg')
    .append('g')
    .attr(
      'transform', 
      'matrix('+scale+' 0 0 -'+scale+' '+dx+' '+dy+')'
    );
```

Toutes les couches doivent simplement aller à l'intérieur de ce groupe pour avoir la bonne transformation.

### 3.5 Classification par Jenks et palette de couleurs

Le dernier problème à résoudre est la classification et la palette de couleurs. En cartographie thématique, il est usuel de recourir à la classification Jenks et d'utiliser une palette de couleurs Colorbrewer.

d3 ne propose pas la classification Jenks, nous devons donc recourir à une autre solution.  Nous allons utiliser la libraire [Classybrew](https://github.com/tannerjt/classybrew) pour cela, qui propose une classification Jenks et en plus les palettes de couleur de Colorbrewer.

Du côté de d3, on peut créer une fonction qui transforme une valeur en couleur:

```javascript
var colorFn = d3.scaleThreshold()
                .domain([0.2, 0.4, 0.5, 0.6, 0.8])
                .range(['#eff3ff','#c6dbef','#9ecae1','#6baed6','#3182bd','#08519c']);
```

qu'on peut utiliser par la suite comme:

```javascript
colorFn(0.56)
```

pour obtenir la couleur corrspondant à la valeur passée.

En conséquence, Classybrew doit nous fournir les limites des classes ainsi que les couleurs associées au classes (notez que le nombre de limites est 1 de moins que le nombre de couleurs).

Classybrew doit être inclut comme libraire externe dans le HTML. Nous utilisons ici un fork du Classybrew original, servi directement depuis Github à travers [rawgit.com](https://rawgit.com), d'où l'URL un peu longue:

```html
<script src="https://cdn.rawgit.com/christiankaiser/classybrew/b8ca77d5a6b7eef5586f5148c36ee937aa1bacfb/build/classybrew.min.js"></script>
```

Puis nous pouvons simplement faire comme suit:

```javascript
// Créer un nouvel objet Classybrew
var brew = new classyBrew();

// Définir la série des données (les valeurs de chaque polygone)
brew.setSeries([40, 40, 42.11, 42.86, 42.87, 42.87, 43.24, ...]);

// Définir le nombre de classes
brew.setNumClasses(6);

// Définir le code de la palette Colorbrewer;
// peut être trouvé dans l'URL sur colorbrewer2.org (dans `scheme=...`)
brew.setColorCode('PuBu');

// Effectuer la classification; cela nous renvoie les limites
// Attention, contient également la valeur min et max
var breaks = brew.classify('jenks');

// Enlever la valeur min et max pour d3
var breaksD3 = breaks.slice(1,6);

// Obtenir les couleurs
var colorPalette = brew.getColors();
```

Ceci nous permet de définir la fonction de couleur de d3 décrite plus haut comme suit:

```javascript
var colorFn = d3.scaleThreshold()
                .domain(breaksD3)
                .range(colorPalette);
```

__Notice concernant le calcul de la classification de Jenks:__ La classification de Jenks nécessite un calcul relativement important, qui est dans ce cas effectué directement dans le navigateur. Suivant l'ordinateur (ou le smartphone ou la tablette) utilisée, ceci peut prendre un peu de temps, notamment avec un nombre important de géométries.

Pour palier à ce problème, on peut adopter deux stratégies:

1. On fait la classification sur un __échantillon aléatoire__ des géométries, échantillon qui est limité dans sa taille (p.ex. 1000 unités au maximum). Ainsi, on n'est pas pris au piège si d'un coup il faut faire la classification sur les 30'000 communes de France. Par contre, il est clair que le résultat de la classification n'est pas nécessairement optimale dans ce cas.

2. Au lieu de __faire le calcul__ dans le navigateur, on le fait __sur un serveur Web__ (p.ex. dans une application Flask) où on contrôle la performance du serveur. En plus, on peut stocker le résultat dans une base de données après le premier calcul, et la deuxième fois utiliser les limites déjà calculées. Il s'agit ici d'une sorte de cache très simple qui peut accéler grandement la performance de la carte.

C'est clairement la deuxième méthode qui est à préférer, mais elle nécessite évidemment la mise en place et la maintenance d'un serveur Web.